# Pipeline

- Data loader: VOC2012
- Network: SSD300
- Loss: MultiboxLoss
- Optimizer
- Tranning, validation

In [64]:
import torch

from lib_all import *

In [65]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cuda:0


# Import 1 số hàm đã tạo

In [66]:
import nbimporter
from data.path import get_path
from data.dataset import MyDataset, collate_fn
from data.processing_data import DataTransform
from data.info_annotation import InfoAnno
from sourc_code.model.model import SSD

# I. Dataloader

Tạo path list trainning và validation

In [67]:
root_path ='../data/data_set/VOCdevkit/VOC2012'
train_images_path, train_annotations_path, val_images_path, val_annotations_path = get_path(root_path)

In [68]:
classes = ['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable','dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

color_mean = (104, 117, 123)
input_size = 300

train_dataset = MyDataset(train_images_path,train_annotations_path, phase = "train", DataTransform = DataTransform(input_size, color_mean), InfoAnno = InfoAnno(classes) )
val_dataset = MyDataset(val_images_path,val_images_path, phase = "val", DataTransform = DataTransform(input_size, color_mean), InfoAnno = InfoAnno(classes) )

# tao data loader
batch_size = 32
train_dataloader = data.DataLoader(train_dataset, batch_size, shuffle=True, collate_fn=collate_fn)
val_dataloader = data.DataLoader(val_dataset, batch_size, shuffle=False, collate_fn=collate_fn)

dataloader_dict = {"train": train_dataloader, "val": val_dataloader}


# II. Network

In [69]:
configs = {
    "num_classes": 21,
    "input_size": 300,
    "aspect_num": [4, 6, 6, 6, 4, 4], #so bbox/ pixel  trong so cho cac sorce tu 1 -> 6
    "feature_maps": [38, 19, 10, 5, 3, 1],
    "steps": [8, 16, 32, 64, 100, 300], # size cua pixel
    "min_size": [30, 60, 111, 162, 213, 264],  #size cua default box
    "max_size": [60, 111, 162, 213, 264, 315], #size cua default box
    "aspect_ratio":[[2],[2,3],[2,3],[2,3],[2],[2]]
}

In [70]:
net = SSD(phase = "train", configs = configs)

load weight

In [71]:
vgg_weigth = torch.load("../data/download/vgg16_reducedfc.pth")

In [72]:
net.vgg.load_state_dict(vgg_weigth)

<All keys matched successfully>